In [29]:
!pip install dadmatools spacy_stanza hazm ipywidgets 

### Read symbols

In [30]:
def create_output(output_type, marker, span, **kwargs):
    defaults = {
        "type": output_type,
        "marker": marker,
        "span": repr(span),
    }
    return {**defaults, **kwargs}


In [31]:
import stanza
import spacy_stanza

stanza.install_corenlp()
stanza.download("fa")
stanza_nlp = spacy_stanza.load_pipeline("fa")


2022-04-15 10:32:30 WARNING: Directory /home/ahura/stanza_corenlp already exists. Please install CoreNLP to a new directory.


2022-04-15 10:32:31 INFO: Downloading default packages for language: fa (Persian)...
2022-04-15 10:32:31 INFO: File exists: /home/ahura/stanza_resources/fa/default.zip.
2022-04-15 10:32:35 INFO: Finished downloading models and saved to /home/ahura/stanza_resources.
2022-04-15 10:32:35 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-04-15 10:32:35 INFO: Use device: gpu
2022-04-15 10:32:35 INFO: Loading: tokenize
2022-04-15 10:32:35 INFO: Loading: mwt
2022-04-15 10:32:35 INFO: Loading: pos
2022-04-15 10:32:35 INFO: Loading: lemma
2022-04-15 10:32:35 INFO: Loading: depparse
2022-04-15 10:32:36 INFO: Done loading processors!


In [32]:
import pandas as pd 
import re 

df = pd.read_csv("symbols_info.csv", index_col=0)
df['Corp. Title'] = df['Corp. Title'].apply(lambda word: re.sub(r"\(.*\)|(ح\.)|(ح\s\.)", "", word))

df.head(n=10)

,Symbol,Corp. Title,TSE URL,Group Name
0,ثملی,بین المللی ساختمان و صنعت ملی,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,انبوه سازي، املاك و مستغلات
1,اخابرح,مخابرات ایران,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,مخابرات
2,شتولیح,تولی‌پرس‌,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,محصولات شيميايي
3,دسبحانح,سبحان دارو,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,مواد و محصولات دارويي
4,وزمین,بانک ایران زمین,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,بانكها و موسسات اعتباري
5,سفارسح,سیمان فارس و خوزستان,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,سيمان، آهك و گچ
6,فنوالح,نورد آلومینیوم‌,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,فلزات اساسي
7,غالبرح,لبنیات ‌کالبر,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,محصولات غذايي و آشاميدني به جز قند و شكر
8,خساپاح,سایپا,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,خودرو و ساخت قطعات
9,مبین,مبین انرژی خلیج فارس,http://tsetmc.com/Loader.aspx?ParTree=151311&i...,عرضه برق، گاز، بخاروآب گرم


In [33]:
texts = [
    # "او یک سیگنال مثبت داد",
    # "نماد برکت افزایش و نماد کگل کاهش یافت.",
    # "برکت امروز اطلاعیه‌ای مهم منتشر میکند.",
    # "نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.",
    # "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    # "نماد برکت امروز عرضه ی اولیه دارد.",
    # "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    # "عرضه های اولیه امروز خوب هستند.",
    # "عرضه اولیه‌های امروز خوب هستند.",
    # "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    # "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    # "نماد برکت امروز عرضه اولیه خیلی خوبی داره.",
    # 'روز چهارشنبه یک دفعه برای خودشون افشا زدن.',
    # "سهام وغدیر و خزر کاهش یافت.",
    # "برای خودشون ی افشایی زدن.",
    # "آ س پ امروز بالا رفت.",
    # "ریزش بازار به دلیل حمله‌ی روسیه هست.",
    "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    # "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    # "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    # "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    # "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    # "شاخص به ۲ میلیون می‌رسه",
    # "آمریکا باعث ریزش بازار شد",
    # "آمریکا موجب ریزش بازار شد",
    # "آمریکا دلیل ریزش بازار شد",
    # "کاهش قیمت سهم عجیب بود",
    # "قیمت زیاد شد",
    # "قیمت زیاد است",
    # "به کتابخانه رفتم.",
    # "به کتابخانه رفت.",
    # "پول در جیب من است.",
    # "سهم قیمتش پایین است",
    # "حضور تو موجب خوشحالی من در هوای بارانی است",
    # "کاهش قیمت سهم خوب است",
    # "کاهش قیمت سهم",
    # "قیمت سهم کاهش یافت",
    # "کاهش زیادی یافت",
    # "بازار روندی متعادل و مثبت دارد",
    # "بازار ده واحد افت کرد",
    # "شاخص ۱۰ واحد رشد کرد",
    # "روز دوشنبه شاخص بورس به صعود ادامه داد و برای دومین روز متوالی افزایش یافت",
    # "نماد با کاهش بیش از ۳ درصدی قیمت همراه بود",
    # "جلسه‌ای برای افزایش کارایی و افزایش نقد شوندگی بازار سرمایه مقرر شد",
    # "جلسه‌ای برای افزایش کارایی و افزایش نقد شوندگی بازار سرمایه مقرر شد",
    # "شاخص با ده واحد افزایش به سقف خود رسید",
    # "کارشناسان، روند متعادل و گاها نزولی برای بازار سهام امروز پیش‌بینی کردند",
    # "کارشناسان، روند نزولی و خوب برای بازار سهام امروز پیش‌بینی کردند",  #Problem, because there is ebham in sent
    # "بورس سه شنبه افت بیش از ۳۳ هزار واحدی داشت",
    # "امروز چند سیگنال برای سهام‌داران وجود دارد", #be nazaram vojood darad ham bayad begin
    # "اصلاح و اعمال تدریجی قوانین بورسی کمک کننده است",
    # "شاخص با ده واحد افزایش به سقف خود رسید",
    # "افزایش ۱۰۰ واحدی قیمت سهام رخ داد",
    # "وعده‌ی اصلاح برخی قوانین مزاحم داده شد",
    # "شاخص بورس ۷ هزار واحد افت کرد",
    # "برخی گمانه‌زنی‌ها از روند رو به مثبت جهش بازار سرمایه خبر می‌دهد",
    # "برخی گمانه‌زنی‌ها از روند مثبت بازار خبر می‌دهند",
    # "برخی گمانه‌زنی‌ها از روند مثبت بازار، خبر می‌دهند",
    # "نماد با کاهش بیش از ۳ درصدی قیمت همراه بود",
    # "روند نزولی ملایم خوب است",
    # "شاخص معاملات ۱۱ درصد افت، ۱۲.۵ درصد رشد و ۲.۲ درصد افزایش یافت", #hazf be gharine?
    # "برکت ۱۱ درصد افت، خزر ۱۲.۵ درصد رشد و کگل ۳ درصد رشد پیدا کرد",
    # "شاخص معاملات ۱۱ درصد افت پیدا کرد، ۱۲.۵ درصد رشد کرد و ۲.۲ درصد افزایش یافت",
    # "خروج سرمایه تاثیر مثبتی بر بازار داشت. اما سهام آ س پ سقوط کرد."
]
events = [
    "سود",
    "ضرر",
    "مثبت",
    "منفی",
    "صعود",
    "نزول",
    "افزایش",
    "کاهش",
    "رشد",
    "ریزش",
    "افشا",
    "افشا ب",
    "افشا الف",
    "اصلاح",
    "افت",
    "نوسان",
    "نوسانات",
    "سقوط",
    "تقسیم سود",
    "تعیین نرخ",
    "عرضه اولیه",
    "صف خرید",
    "صف فروش",
    "خروج سرمایه",
    "افزایش سرمایه"
    "خروج پول",
    "مجمع عمومی",
]
announcements = [
    "اطلاعیه",
    "گزارش",
    "اعلامیه",
    "اعلانیه"
]
analyse = [
    "سهم رانتی",
    "تحلیل تکنیکال",
    "تحلیل فاندامنتال",
    "کندل استیک",
    "سیگنال",
    "تکنیکال",
    "کندل",
    "تیک",
    "سرشانه",
    "مقاومت",
    "حمایت",
    "کراس",
    "واگرایی",
    "اندیکاتور",
]
characters =[
    "بازیگر",
    "حقیقی",
    "حقوقی",
    "بازی‌گردان",
    "نوسان‌گیر",
]
# reason_propositions = [
#     "دلیل",
#     "باعث",
#     "موجب",
#     "برای"
# ]

In [34]:
from dadmatools.models.normalizer import Normalizer

normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="<URL",
    replace_mobile_number_with="<MOBILE_NUMBER>",
    replace_emoji_with="<EMOJI>",
    replace_home_number_with="<HOME_NUMBER>",
)


In [35]:
from spacy.matcher import Matcher
matcher: Matcher = Matcher(stanza_nlp.vocab, validate=True)

def add_patterns(matcher, keywords, class_name):
  keyword_patterns = []
  for keyword in keywords:
      keyword_tokens = stanza_nlp.tokenizer(keyword)

      pre_det = ["این", "هر", "چند", "چندین", "آن", "همین", "همان", "چنین", "چنین"]
      
      if len(keyword_tokens) == 1:
          token1_keyword = keyword_tokens[0].text
          pattern = [
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": token1_keyword}},
          ]

      elif len(keyword_tokens) == 2:
          token1_keyword = keyword_tokens[0].text
          token2_keyword = keyword_tokens[1].text
          pattern = [
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": token1_keyword}},
              {'TEXT': {"IN": ["ای", "ی", "ها", "های"]}, "OP": "?"},
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": token2_keyword}},
          ]
          
      keyword_patterns.append(pattern)

  matcher.add(class_name, keyword_patterns, greedy="LONGEST")

SYMBOL = "SYMBOL"
EVENT = "EVENT"
ANALYSE = "ANALYSE"
ANNOUNCE = "ANNOUNCE"
CHARACTERS = "CHARACTERS"

add_patterns(matcher, events, "EVENT")
add_patterns(matcher, analyse, "ANALYSE")
add_patterns(matcher, announcements, "ANNOUNCE")
add_patterns(matcher, characters, "CHARACTERS")

In [36]:
symbols = df["Symbol"].tolist()

In [39]:
import re
from spacy.tokens import Span
from spacy import displacy

def convert_subtree_to_str(token, not_in=[], remove_ADP=False):
    extracted_event = ''
    start = None
    end = None
    for x in token.subtree:
        if x.pos_ in not_in:
            continue
        if remove_ADP and x.pos_ == 'ADP':
            if x.head.text == token.text:
                continue

        extracted_event += str(x) + ' '
        if start is None:
            start = x.idx
        end = x.idx + len(x.text)
        
    extracted_event = extracted_event[:-1]
    return extracted_event, start, end

TRANSLATE = {
    SYMBOL: 'نماد',
    EVENT: 'واقعه',
    ANNOUNCE:'اعلان',
    ANALYSE:'تحلیل',
    CHARACTERS:'شخصیت'
}

def create_output(output_type, marker, span, **kwargs):
    output_type = TRANSLATE[output_type]
    defaults = {
        "type":output_type, 
        "marker":marker, 
        "span":repr(span),
        }
    
    return {
        **defaults,
        **kwargs
    }

def create_spans(label):

    term_spans = list(map(
        lambda match: Span(doc, match[1], match[2], label=stanza_nlp.vocab.strings[match[0]]),
        matcher(doc)
        ))
    
    term_spans = list(filter(
        lambda span: span is not None,
        term_spans
        ))
    
    return term_spans


symbols_expression = "|".join(symbols)

for text_index, text in enumerate(texts):
    text = normalizer.normalize(text)

    print(f'Text {text_index + 1}...')
    print(f"Normalized Text: {text}")

    doc = stanza_nlp(text)
    senetences = doc.sents
    
    symbol_spans = list(map(
        lambda match: doc.char_span(*match.span(), label=SYMBOL),
        re.finditer(symbols_expression, text)
        ))
    
    symbol_spans = list(filter(
        lambda span: span is not None,
        symbol_spans
        ))

    term_spans = list(map(
        lambda match: Span(doc, match[1], match[2], label=stanza_nlp.vocab.strings[match[0]]),
        matcher(doc)
        ))
    
    term_spans = list(filter(
        lambda span: span is not None,
        term_spans
        ))

   
    spans = symbol_spans + term_spans
    doc.set_ents(spans)
    
    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in symbol_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in term_spans:
            retokenizer.merge(span, attrs)

    print('Entities: ...')
    displacy.render(doc, style='ent', jupyter=True)
    
    print('Dependency Tree: ...')
    displacy.render(doc, style='dep', jupyter=True)

    symbol_ents = list(filter(lambda ent: ent.label_ == SYMBOL,
                     doc.ents))
    
    for symbol_ent in symbol_ents:
        token = symbol_ent[0]
        start = token.idx
        end = token.idx + len(token.text)
        print(create_output(SYMBOL, token.text, (start,end)))

    
    term_ents = list(filter(lambda ent: ent.label_ != SYMBOL,
                     doc.ents))
    
    from spacy.symbols import VERB, NOUN
    
    for term_ent in term_ents:
        token = term_ent[0]
        
        if token.dep_ == 'compound:lvc':
            print('type is: coumpound:lvd')
            print('parent is: ', token.head)
            if token.head.pos == VERB:
                print('parent was a verb')
                print('children of parent: ', list(token.head.children))
                subject = None
                for child in token.head.children:
                    if child.dep_ == 'nsubj':
                        print('Found a subject, outputing subtree of subject as subject')
                        subject, start_subj, end_subj = convert_subtree_to_str(child)
                        break
                    else:
                        print('NOT IMP - child dep is: ', child.dep_)
                        pass
                else:
                  print('didn\'t found any subj')

                start = token.idx
                end = token.idx + len(token.text) + 1 + len(token.head.text)
                extracted_event = token.text + ' ' + token.head.text #TODO: check
                if subject:
                    print(create_output(token.ent_type_, extracted_event, (start,end), subject=subject))
                else:
                    print(create_output(token.ent_type_, extracted_event, (start,end)))

            # elif token.dep_ == 'nmod':
            #     print('type is: nmod')
            #     print('parent is: ', token.head)

            #     print('finding parent that is not nmod')
            #     main_noun = token
            #     while main_noun.dep_ == 'nmod' and main_noun.head.pos == NOUN:
            #       main_noun = main_noun.head
            #     if main_noun.pos != NOUN:
            #       continue

            #     print('Found a Noun parent that is not nmod')
            #     extracted_event, start, end = convert_subtree_to_str(token.head, not_in=['ADP'])

        elif token.dep_ == 'nmod' or token.dep_ == 'amod':
            print('type is: ', token.dep_)
            print('parent is: ', token.head)

            print('finding parent that is not nmod or amod')
            main_noun = token
            # print('1', main_noun, main_noun.dep_, main_noun.head.pos_)
            while (main_noun.dep_ == 'nmod' or main_noun.dep_ == 'amod') and main_noun.head.pos == NOUN:
                main_noun = main_noun.head
                # print('2', main_noun, main_noun.dep_, main_noun.head.pos_)
            if main_noun.pos != NOUN:
                continue

            print('Found a Noun parent that is not nmod or amod')
            extracted_event, start, end = convert_subtree_to_str(main_noun, remove_ADP=True)
            # extracted_event, start, end = convert_subtree_to_str(main_noun, not_in=['ADP'])

            print(create_output(token.ent_type_, extracted_event, (start,end)))

        else:
            print('type is: other')
            extracted_event, start, end = convert_subtree_to_str(token, remove_ADP=True)

            # print(text[start:end])
            print(create_output(token.ent_type_, extracted_event, (start,end)))

           

Text 1...
Normalized Text: فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.
Entities: ...


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


Dependency Tree: ...


type is: other
{'type': 'واقعه', 'marker': 'یه اصلاح قیمتی و کمی ریزش', 'span': '(7, 32)'}
type is: other
{'type': 'واقعه', 'marker': 'و کمی ریزش', 'span': '(22, 32)'}
